In [ ]:
import os, sys
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import time
import shutil
import collections
from pathlib import Path

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Loading packages

In [ ]:
import sys
from pathlib import Path

here_path = Path().resolve()
repo_path = here_path.parents[1]
sys.path.append(str(repo_path))

In [ ]:
from py.utils import verifyDir, verifyFile, verifyType

In [ ]:
from py.config import Config

cfg = Config()

np.random.seed(cfg.RANDOM_STATE)
cfg.DATA_PATH, cfg.MODEL_PATH

In [ ]:
QSCORE_PATH=f"{cfg.DATA_PATH}pp2/{cfg.SCORING_METHOD}/{cfg.PLACE_LEVEL}/"
IMAGES_PATH = f"{cfg.DATA_PATH}pp2/images/"

CITYSCAPES_DIR = f"{cfg.DATA_PATH}CityScapes/"
ADE20K_DIR = f"{cfg.DATA_PATH}ADE20k/"

In [ ]:
verifyDir(CITYSCAPES_DIR)

#### Loading segmentations

In [ ]:
from py.models.segmentation.cnn import ConvMaskClassifier

model = ConvMaskClassifier(dataset="cityscapes", load_model=False)

In [ ]:
cityscape_data_dict = model._get_color_dict()
cityscape_data_dict

In [ ]:
from py.utils.color import rgb2hex

cityscapes_df = pd.DataFrame(cityscape_data_dict, columns=['class_id', 'class_name', "RGB_color"])
cityscapes_df["main_class"] = cityscapes_df["class_name"].apply(lambda x: "_".join(x.split()))
cityscapes_df["hex_color"] = cityscapes_df["RGB_color"].apply(rgb2hex)
cityscapes_df

In [ ]:
isthing_df = pd.read_csv(f"{ADE20K_DIR}ade20k_categories.csv", sep=";", low_memory=False)
isthing_df.drop(columns=["RGB_color", "hex_color"], inplace=True)
isthing_df

In [ ]:
cde_df = pd.merge(cityscapes_df, isthing_df[["main_class", "isthing", "group_name"]].copy(), how="left", on="main_class")
# Completing group names
cde_df.loc[(cde_df["main_class"]=="background"), "isthing"] = -1
cde_df.loc[(cde_df["main_class"]=="background"), "group_name"] = "background"

cde_df.loc[(cde_df["main_class"]=="traffic_light"), "isthing"] = 1
cde_df.loc[(cde_df["main_class"]=="traffic_light"), "group_name"] = "city_elements"

cde_df.loc[(cde_df["main_class"]=="traffic_sign"), "isthing"] = 1
cde_df.loc[(cde_df["main_class"]=="traffic_sign"), "group_name"] = "city_elements"

cde_df.loc[(cde_df["main_class"]=="vegetation"), "isthing"] = 0
cde_df.loc[(cde_df["main_class"]=="vegetation"), "group_name"] = "vegetation"

cde_df.loc[(cde_df["main_class"]=="terrain"), "isthing"] = 0
cde_df.loc[(cde_df["main_class"]=="terrain"), "group_name"] = "floor"

cde_df.loc[(cde_df["main_class"]=="rider"), "isthing"] = 1
cde_df.loc[(cde_df["main_class"]=="rider"), "group_name"] = "human"

cde_df.loc[(cde_df["main_class"]=="train"), "isthing"] = 1
cde_df.loc[(cde_df["main_class"]=="train"), "group_name"] = "terrain_vehicle"

# Groups
cde_df.loc[(cde_df["group_name"]=="background"), "RGB_color_group"] = "(0, 0, 0)"
cde_df.loc[(cde_df["group_name"]=="terrain_vehicle"), "RGB_color_group"] = "(255, 0, 20)"
cde_df.loc[(cde_df["group_name"]=="floor"), "RGB_color_group"] = "(140, 140, 140)"
cde_df.loc[(cde_df["group_name"]=="construction"), "RGB_color_group"] = "(180, 120, 120)"
cde_df.loc[(cde_df["group_name"]=="city_elements"), "RGB_color_group"] = "(0, 71, 255)"
cde_df.loc[(cde_df["group_name"]=="vegetation"), "RGB_color_group"] = "(4, 200, 3)"
cde_df.loc[(cde_df["group_name"]=="sky"), "RGB_color_group"] = "(6, 230, 230)"
cde_df.loc[(cde_df["group_name"]=="human"), "RGB_color_group"] = "(150, 5, 61)"

cde_df["hex_color_group"] = cde_df["RGB_color_group"].apply(rgb2hex)
cde_df.sort_values(by="group_name", inplace=True)
cde_df.reset_index(inplace=True, drop=True)
cde_df

In [ ]:
cde_df.to_csv(f"{CITYSCAPES_DIR}cityscapes_categories.csv", sep=";", index=False)